In [ ]:
!apt-get install -y build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
# !pip install lightfm
import pandas as pd
from scipy import sparse
import lightfm
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from lightfm.data import Dataset
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 138.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 143.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
  Attempting uninstall: sc

In [ ]:
import ast
import numpy as np
# 1. Load data
users = pd.read_csv('user_data.csv')
posts = pd.read_csv('final_post_data.csv')
interactions = pd.read_csv('user_item_interaction.csv')

In [ ]:
users

,Unnamed: 0,user_id,gender,age,city,interests,education_level,active_hours
0,0,1,Female,30,Bengaluru,"['Movies', 'Real Estate', 'Marketing', 'Parent...",Undergraduate,16
1,1,2,Male,55,Chennai,"['History', 'Parenting', 'Fashion', 'DIY & Cra...",Undergraduate,12
2,2,3,Female,58,Delhi,"['Education', 'DIY & Crafts', 'Finance', 'Poli...",Undergraduate,4
3,3,4,Male,25,Mumbai,"['Music', 'Food', 'Gaming', 'History', 'Techno...",Undergraduate,16
4,4,5,Female,59,Bengaluru,"['Marketing', 'Business', 'Psychology', 'Music...",High School,17
...,...,...,...,...,...,...,...,...
495,495,496,Female,40,Hyderabad,"['Spirituality', 'News', 'Business', 'History'...",High School,8
496,496,497,Male,37,Hyderabad,"['Automotive', 'Books', 'Technology', 'History...",Undergraduate,5
497,497,498,Female,36,Chennai,"['DIY & Crafts', 'Gaming', 'Business', 'Parent...",High School,13
498,498,499,Female,46,Mumbai,"['Parenting', 'Law', 'Food', 'Music', 'Photogr...",High School,17


In [ ]:
posts

,post_id,title,category,comments,likes,saves,shares,views
0,1,10 Study Hacks Backed by Neuroscience (Do They...,Education,10,7,3,4,30
1,2,10 Easy DIY Crafts to Try This Weekend,DIY & Crafts,11,13,10,11,51
2,3,I Quit School for 6 Months to Self-Teach (Resu...,Education,4,4,9,9,38
3,4,10 Ancient Recipes Modern Chefs Cant Replicat...,Food,10,10,7,5,36
4,5,The Political Cost of Climate Inaction in Deve...,Politics,5,11,5,10,41
...,...,...,...,...,...,...,...,...
4995,4996,The One Parenting Tip That Changed Everything,Parenting,7,6,8,6,37
4996,4997,Teaching Volcanology by Visiting Active Erupti...,Education,6,10,7,9,37
4997,4998,The Law Behind Voter ID Controversies,Law,5,5,8,9,38
4998,4999,Psychology of Persuasion: How Marketers Get In...,Psychology,6,10,4,11,37


In [ ]:
interactions

,user_id,post_id,timestamp,interaction_strength
0,1,1823,2024-10-22 00:00:00,2
1,1,3960,2024-10-26 02:00:00,4
2,1,4943,2024-02-08 06:00:00,3
3,1,4323,2024-06-30 01:00:00,3
4,1,1150,2024-09-02 22:00:00,4
...,...,...,...,...
199995,500,982,2024-09-01 19:00:00,4
199996,500,405,2024-04-26 13:00:00,5
199997,500,3799,2024-06-05 13:00:00,4
199998,500,4782,2024-09-02 00:00:00,2


In [ ]:
# 2. Prepare user and item features
mlb = MultiLabelBinarizer()

# User features
users['interests'] = users['interests'].apply(ast.literal_eval)  # Convert string to list

In [ ]:
def bucket_age(age):
    if age < 20:
        return 'age_<20'
    elif age < 30:
        return 'age_20s'
    elif age < 40:
        return 'age_30s'
    else:
        return 'age_40+'

def bucket_hours(h):
    if h < 3:
        return 'hours_low'
    elif h < 6:
        return 'hours_medium'
    else:
        return 'hours_high'

users['age_bucket'] = users['age'].apply(bucket_age)
users['hours_bucket'] = users['active_hours'].apply(bucket_hours)

In [ ]:
users

,Unnamed: 0,user_id,gender,age,city,interests,education_level,active_hours,age_bucket,hours_bucket
0,0,1,Female,30,Bengaluru,"[Movies, Real Estate, Marketing, Parenting, Tr...",Undergraduate,16,age_30s,hours_high
1,1,2,Male,55,Chennai,"[History, Parenting, Fashion, DIY & Crafts, Ho...",Undergraduate,12,age_40+,hours_high
2,2,3,Female,58,Delhi,"[Education, DIY & Crafts, Finance, Politics, S...",Undergraduate,4,age_40+,hours_medium
3,3,4,Male,25,Mumbai,"[Music, Food, Gaming, History, Technology, Fas...",Undergraduate,16,age_20s,hours_high
4,4,5,Female,59,Bengaluru,"[Marketing, Business, Psychology, Music, Art, ...",High School,17,age_40+,hours_high
...,...,...,...,...,...,...,...,...,...,...
495,495,496,Female,40,Hyderabad,"[Spirituality, News, Business, History, Law, M...",High School,8,age_40+,hours_high
496,496,497,Male,37,Hyderabad,"[Automotive, Books, Technology, History, Healt...",Undergraduate,5,age_30s,hours_medium
497,497,498,Female,36,Chennai,"[DIY & Crafts, Gaming, Business, Parenting, Ho...",High School,13,age_30s,hours_high
498,498,499,Female,46,Mumbai,"[Parenting, Law, Food, Music, Photography, Mar...",High School,17,age_40+,hours_high


In [ ]:
user_features_tmp = pd.get_dummies(users[['age_bucket', 'hours_bucket', 'gender', 'city']])

In [ ]:
user_features_tmp

,age_bucket_age_20s,age_bucket_age_30s,age_bucket_age_40+,age_bucket_age_<20,hours_bucket_hours_high,hours_bucket_hours_low,hours_bucket_hours_medium,gender_Female,gender_Male,city_Ahmedabad,city_Bengaluru,city_Chennai,city_Delhi,city_Hyderabad,city_Mumbai,city_Pune
0,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False
1,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False,False
2,False,False,True,False,False,False,True,True,False,False,False,False,True,False,False,False
3,True,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False
4,False,False,True,False,True,False,False,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,False,False,True,False,True,False,False,True,False,False,False,False,False,True,False,False
496,False,True,False,False,False,False,True,False,True,False,False,False,False,True,False,False
497,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False
498,False,False,True,False,True,False,False,True,False,False,False,False,False,False,True,False


In [ ]:
user_interest_features = pd.DataFrame(mlb.fit_transform(users['interests']), columns=[f'interest_{i}' for i in mlb.classes_])
user_features = pd.concat([users['user_id'], user_features_tmp, user_interest_features], axis=1)

In [ ]:
user_interest_features

,interest_Art,interest_Automotive,interest_Books,interest_Business,interest_DIY & Crafts,interest_Education,interest_Environment,interest_Fashion,interest_Finance,interest_Food,...,interest_Photography,interest_Politics,interest_Psychology,interest_Real Estate,interest_Relationships,interest_Science,interest_Spirituality,interest_Sports,interest_Technology,interest_Travel
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,1,0,0,1,0,...,0,1,1,0,0,1,1,0,1,0
3,0,0,0,1,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,0,1,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
496,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
497,0,0,0,1,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [ ]:
user_features

,user_id,age_bucket_age_20s,age_bucket_age_30s,age_bucket_age_40+,age_bucket_age_<20,hours_bucket_hours_high,hours_bucket_hours_low,hours_bucket_hours_medium,gender_Female,gender_Male,...,interest_Photography,interest_Politics,interest_Psychology,interest_Real Estate,interest_Relationships,interest_Science,interest_Spirituality,interest_Sports,interest_Technology,interest_Travel
0,1,False,True,False,False,True,False,False,True,False,...,0,0,0,1,0,0,0,0,0,1
1,2,False,False,True,False,True,False,False,False,True,...,0,0,0,0,0,0,0,0,0,0
2,3,False,False,True,False,False,False,True,True,False,...,0,1,1,0,0,1,1,0,1,0
3,4,True,False,False,False,True,False,False,False,True,...,0,1,0,0,0,0,0,0,1,0
4,5,False,False,True,False,True,False,False,True,False,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,False,False,True,False,True,False,False,True,False,...,0,1,0,0,0,0,1,0,1,0
496,497,False,True,False,False,False,False,True,False,True,...,0,0,0,0,0,0,0,0,1,0
497,498,False,True,False,False,True,False,False,True,False,...,0,0,1,1,0,0,0,0,0,0
498,499,False,False,True,False,True,False,False,True,False,...,1,0,0,0,0,0,0,1,0,0


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Download the necessary Punkt tab resource
nltk.download('punkt_tab')

!pip install emoji
import re
import emoji


def clean_text(text):
    # Convert non-string values to an empty string
    if not isinstance(text, str):
        text = ''

    # Remove emojis first
    text = emoji.replace_emoji(text, replace='')

    # Keep hashtags but remove the '#' symbol
    text = re.sub(r'#(\w+)', r'\1', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions
    text = re.sub(r'@\w+', '', text)

    # Remove special characters and numbers (except hashtags)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text

posts['title'] = posts['title'].apply(clean_text)

# Tokenize titles
posts['title_tokens'] = posts['title'].apply(lambda x: word_tokenize(x.lower()))

# Train Word2Vec using Skipgram
w2v_model = Word2Vec(sentences=posts['title_tokens'], vector_size=50, window=5, min_count=1, workers=4,sg=1)


def get_title_embedding(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(w2v_model.vector_size)

posts['title_embedding'] = posts['title_tokens'].apply(get_title_embedding)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 12.7 MB/s eta 0:00:00


In [ ]:
posts['title_embedding']

,title_embedding
0,"[-0.038508046, -0.007956684, 0.030371577, 0.00..."
1,"[-0.0673996, -0.013721117, 0.010100231, -0.010..."
2,"[-0.05640791, -0.013755661, 0.020952959, -0.00..."
3,"[-0.022202088, -0.0034048648, 0.035281423, 0.0..."
4,"[-0.017963508, -0.011636413, 0.0529897, 0.0205..."
...,...
4995,"[-0.023286676, -0.012447139, 0.06796871, 0.043..."
4996,"[-0.028882166, -0.0038022136, 0.028917016, 0.0..."
4997,"[-0.0147751225, -0.011358953, 0.048414987, 0.0..."
4998,"[-0.06159194, -0.011002749, 0.015608246, -0.00..."


In [ ]:
# Item features
posts['category'] = posts['category'].astype(str)

In [ ]:
# Making Item Features
item_features = pd.concat([
    posts[['post_id']],
    pd.get_dummies(posts['category'], prefix='category'),
    posts[['likes', 'comments', 'shares','views','saves']]
], axis=1)

In [ ]:
item_features

,post_id,category_Art,category_Automotive,category_Books,category_Business,category_DIY & Crafts,category_Education,category_Environment,category_Fashion,category_Finance,...,category_Science,category_Spirituality,category_Sports,category_Technology,category_Travel,likes,comments,shares,views,saves
0,1,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,7,10,4,30,3
1,2,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,13,11,11,51,10
2,3,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,4,4,9,38,9
3,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,10,10,5,36,7
4,5,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,11,5,10,41,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,6,7,6,37,8
4996,4997,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,10,6,9,37,7
4997,4998,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,5,5,9,38,8
4998,4999,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,10,6,11,37,4


In [ ]:
# Convert each vector into a DataFrame row with 50 columns
embedding_df = pd.DataFrame(posts['title_embedding'].to_list(), columns=[f'embedding_{i}' for i in range(w2v_model.vector_size)])

In [ ]:
embedding_df

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,-0.038508,-0.007957,0.030372,0.006491,-0.145533,-0.259813,0.228094,0.353031,-0.429692,-0.310738,...,0.303205,-0.011244,-0.028170,-0.047989,0.510834,0.088766,-0.095103,-0.249288,0.101260,0.226898
1,-0.067400,-0.013721,0.010100,-0.010414,-0.126577,-0.232655,0.272048,0.358384,-0.413135,-0.337985,...,0.316765,0.004676,-0.064430,-0.065487,0.526553,0.091290,-0.084802,-0.321625,0.088965,0.204001
2,-0.056408,-0.013756,0.020953,-0.005211,-0.118640,-0.220977,0.229874,0.323949,-0.372176,-0.295409,...,0.283320,0.004537,-0.040160,-0.056073,0.467055,0.085422,-0.078004,-0.270736,0.080228,0.194372
3,-0.022202,-0.003405,0.035281,0.015076,-0.124116,-0.224258,0.196972,0.297391,-0.354277,-0.250236,...,0.255512,-0.004676,-0.029620,-0.034058,0.441229,0.079222,-0.078134,-0.205687,0.092847,0.207023
4,-0.017964,-0.011636,0.052990,0.020572,-0.156147,-0.261292,0.236217,0.317114,-0.366503,-0.273909,...,0.240466,0.006710,-0.023364,-0.066992,0.486255,0.118498,-0.071069,-0.193781,0.113638,0.250105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-0.023287,-0.012447,0.067969,0.043709,-0.186263,-0.277683,0.266679,0.368794,-0.444025,-0.314112,...,0.312321,-0.004243,-0.044162,-0.051657,0.553218,0.115967,-0.115761,-0.242501,0.115056,0.279522
4996,-0.028882,-0.003802,0.028917,0.008324,-0.090075,-0.169178,0.151196,0.232705,-0.284829,-0.193926,...,0.192084,0.002417,-0.023563,-0.026809,0.338940,0.075280,-0.065011,-0.158613,0.066325,0.147844
4997,-0.014775,-0.011359,0.048415,0.033707,-0.138665,-0.221846,0.192889,0.265862,-0.310967,-0.223242,...,0.208118,0.009844,-0.019770,-0.054913,0.395700,0.101769,-0.079428,-0.156294,0.105920,0.228513
4998,-0.061592,-0.011003,0.015608,-0.008420,-0.120206,-0.237044,0.264211,0.333231,-0.370312,-0.323016,...,0.277179,0.009434,-0.040207,-0.080138,0.485065,0.103428,-0.052863,-0.265812,0.090680,0.183793


In [ ]:
# Add post_id
embedding_df['post_id'] = posts['post_id'].values

In [ ]:
# Merge with embeddings on post_id
item_features = item_features.merge(embedding_df, on='post_id')

In [ ]:
item_features

,post_id,category_Art,category_Automotive,category_Books,category_Business,category_DIY & Crafts,category_Education,category_Environment,category_Fashion,category_Finance,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,1,False,False,False,False,False,True,False,False,False,...,0.303205,-0.011244,-0.028170,-0.047989,0.510834,0.088766,-0.095103,-0.249288,0.101260,0.226898
1,2,False,False,False,False,True,False,False,False,False,...,0.316765,0.004676,-0.064430,-0.065487,0.526553,0.091290,-0.084802,-0.321625,0.088965,0.204001
2,3,False,False,False,False,False,True,False,False,False,...,0.283320,0.004537,-0.040160,-0.056073,0.467055,0.085422,-0.078004,-0.270736,0.080228,0.194372
3,4,False,False,False,False,False,False,False,False,False,...,0.255512,-0.004676,-0.029620,-0.034058,0.441229,0.079222,-0.078134,-0.205687,0.092847,0.207023
4,5,False,False,False,False,False,False,False,False,False,...,0.240466,0.006710,-0.023364,-0.066992,0.486255,0.118498,-0.071069,-0.193781,0.113638,0.250105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,False,False,False,False,False,False,False,False,False,...,0.312321,-0.004243,-0.044162,-0.051657,0.553218,0.115967,-0.115761,-0.242501,0.115056,0.279522
4996,4997,False,False,False,False,False,True,False,False,False,...,0.192084,0.002417,-0.023563,-0.026809,0.338940,0.075280,-0.065011,-0.158613,0.066325,0.147844
4997,4998,False,False,False,False,False,False,False,False,False,...,0.208118,0.009844,-0.019770,-0.054913,0.395700,0.101769,-0.079428,-0.156294,0.105920,0.228513
4998,4999,False,False,False,False,False,False,False,False,False,...,0.277179,0.009434,-0.040207,-0.080138,0.485065,0.103428,-0.052863,-0.265812,0.090680,0.183793


In [ ]:
# Normalizing the features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_normalize = ['views','saves','likes', 'comments', 'shares'] + [f'embedding_{i}' for i in range(w2v_model.vector_size)]
item_features[columns_to_normalize] = scaler.fit_transform(item_features[columns_to_normalize])

In [ ]:
# 3. Build dataset
dataset = Dataset()

In [ ]:
dataset.fit(users['user_id'],
            posts['post_id'],
            user_features=user_features.columns[1:],
            item_features=item_features.columns[1:]
            )

In [ ]:
(interactions_matrix, weights) = dataset.build_interactions(
    [(row['user_id'], row['post_id'], row['interaction_strength']) for _, row in interactions.iterrows()]
)

In [ ]:
interactions_matrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
weights.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 2., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 3., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
user_features_matrix = dataset.build_user_features(
    [(row['user_id'], [feat for feat in user_features.columns[1:] if row[feat] != 0]) for _, row in user_features.iterrows()]
)

In [ ]:
item_features_matrix = dataset.build_item_features(
    [(row['post_id'],
      [feat for feat in item_features.columns[1:] if row[feat] != 0])
     for _, row in item_features.iterrows()]
)

In [ ]:
item_features_matrix.shape

(5000, 5085)

In [ ]:
# Train the model
model = LightFM(loss='warp', no_components=70, learning_schedule='adadelta', learning_rate=0.05, item_alpha=1e-6, user_alpha=1e-6)
model.fit(interactions_matrix,
          user_features=user_features_matrix,
          item_features=item_features_matrix,
          sample_weight=weights,
          epochs=20, num_threads=4)

KeyboardInterrupt: 

In [ ]:
import numpy as np

def get_unseen_items(user_id, interactions_matrix):
    # Get all items the user has interacted with
    seen_items = interactions_matrix.tocsr()[user_id].indices

    # All items
    all_items = np.arange(interactions_matrix.shape[1])

    # Filter out seen items
    unseen_items = np.setdiff1d(all_items, seen_items)
    return unseen_items

In [ ]:
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

# Basic evaluation (Precision@K and AUC)
precision = precision_at_k(model, interactions_matrix, k=10,
                           user_features=user_features_matrix,
                           item_features=item_features_matrix).mean()

auc = auc_score(model, interactions_matrix,
                user_features=user_features_matrix,
                item_features=item_features_matrix).mean()

# Intralist-Diversity : pairwise cosine dissimilarity of recommended items
def compute_diversity(model, item_features_matrix, top_k=10):
    item_embeddings = model.item_embeddings
    item_norm = item_embeddings / np.linalg.norm(item_embeddings, axis=1, keepdims=True) # Normalizing
    similarity = cosine_similarity(item_norm)
    diversity = []

    for user_id in range(interactions_matrix.shape[0]):
        unseen_items = get_unseen_items(user_id, interactions_matrix)
        scores = model.predict(user_id, unseen_items,
                               user_features=user_features_matrix,
                               item_features=item_features_matrix)
        top_items = unseen_items[np.argsort(-scores)[:top_k]]

        # Diversity: 1 - avg cosine similarity among top_items
        sim_matrix = similarity[np.ix_(top_items, top_items)]
        avg_sim = (np.sum(sim_matrix) - top_k) / (top_k * (top_k - 1))
        diversity.append(1 - avg_sim)

    return np.mean(diversity)

diversity_score = compute_diversity(model, item_features_matrix, top_k=10)

# Serendipity: reward unusual but relevant items
def compute_serendipity(model, interactions_matrix, top_k=10):
    serendipity = []
    item_popularity = np.array(interactions_matrix.sum(axis=0)).flatten()

    for user_id in range(interactions_matrix.shape[0]):
        unseen_items = get_unseen_items(user_id, interactions_matrix)
        scores = model.predict(user_id, unseen_items,
                               user_features=user_features_matrix,
                               item_features=item_features_matrix)
        top_items = unseen_items[np.argsort(-scores)[:top_k]]

        pop_scores = item_popularity[top_items]

        # Calculating Serendipity Score
        serendipity_score = 1 - np.mean(pop_scores / (pop_scores.max() + 1e-10))
        serendipity.append(serendipity_score)

    return np.mean(serendipity)

serendipity_score = compute_serendipity(model, interactions_matrix, top_k=10)

# Print evaluation
print(f"Precision@10: {precision:.4f}")
print(f"AUC Score: {auc:.4f}")
print(f"Diversity Score: {diversity_score:.4f}")
print(f"Serendipity Score: {serendipity_score:.4f}")

Precision@10: 0.6870
AUC Score: 0.8722
Diversity Score: 0.8058
Serendipity Score: 0.1778


In [ ]:
def recommend_top_n(user_id, model, interactions_matrix, user_features_matrix, item_features_matrix, n=10):
    unseen_items = get_unseen_items(user_id, interactions_matrix)

    # Predict scores for all unseen items
    scores = model.predict(user_id, unseen_items,
                           user_features=user_features_matrix,
                           item_features=item_features_matrix)

    top_indices = np.argsort(-scores)[:n]
    top_items = unseen_items[top_indices]

    return top_items  # These are post_ids (index positions in your matrix)

# Example usage:
user_id = 123  # Change as per your user index
top_recommendations = recommend_top_n(user_id, model, interactions_matrix, user_features_matrix, item_features_matrix, n=10)

print("User Interests: ")
print(users.iloc[user_id]['interests'])
print()
print("Top-N Recommended Post IDs for User:")
for id in top_recommendations:
    print()
    print(posts.iloc[id]['title'])
    print(f"Category: {posts.iloc[id]['category']}")

User Interests: 
['Automotive', 'News', 'Psychology', 'Movies', 'Health & Fitness', 'Parenting', 'Music', 'Photography', 'Education']

Top-N Recommended Post IDs for User:

refugee crisis escalates at border
Category: News

tips for parenting strongwilled kids
Category: Parenting

rise of female directors in cinema
Category: Movies

how to avoid overparenting
Category: Parenting

do you know anyone who is in my shoes and just want to express
Category: Music

healthy grocery shopping on a budget
Category: Health & Fitness

how inception was made vfx breakdown
Category: Movies

creating a calm down corner at home
Category: Parenting

breaking news major airline files for bankruptcy
Category: News

nissan gtr offroad test can it survive
Category: Automotive


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
import numpy as np

# Convert interaction matrix to item-user matrix (transpose)
item_user_matrix = interactions_matrix.T.tocsr()  # shape: [n_items, n_users]

# Compute cosine similarity between items
item_similarity = cosine_similarity(item_user_matrix)

# Mapping IDs back to original post_ids
_, _, item_id_map, _ = dataset.mapping()
reverse_item_map = {v: k for k, v in item_id_map.items()}

# Recommend items similar to one the user liked
def item_knn_recommend(user_id, top_k=10):
    if user_id not in dataset.mapping()[0]:
        raise ValueError("User ID not in model.")

    user_index = dataset.mapping()[0][user_id]
    user_interactions = interactions_matrix.tocsr()[user_index] #efficient row access

    liked_items = user_interactions.indices
    scores = np.zeros(item_similarity.shape[0])

    for item_idx in liked_items:
        scores += item_similarity[item_idx] #Adds up similarity scores from all items the user liked.

    # Remove already liked items
    scores[liked_items] = -1

    top_items = np.argsort(-scores)[:top_k]
    return [reverse_item_map[i] for i in top_items]

# Example: Get item-based recommendations
recommended_ids = item_knn_recommend(user_id=123, top_k=10)
print("Recommended post IDs:")
for id in top_recommendations:
    print()
    print(posts.iloc[id]['title'])
    print(f"Category: {posts.iloc[id]['category']}")


Recommended post IDs:

refugee crisis escalates at border
Category: News

tips for parenting strongwilled kids
Category: Parenting

rise of female directors in cinema
Category: Movies

how to avoid overparenting
Category: Parenting

do you know anyone who is in my shoes and just want to express
Category: Music

healthy grocery shopping on a budget
Category: Health & Fitness

how inception was made vfx breakdown
Category: Movies

creating a calm down corner at home
Category: Parenting

breaking news major airline files for bankruptcy
Category: News

nissan gtr offroad test can it survive
Category: Automotive


In [ ]:
import pickle

# Save LightFM model
with open('lightfm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save user features matrix
with open('user_features_matrix.pkl', 'wb') as f:
    pickle.dump(user_features_matrix, f)

# Save item features matrix
with open('item_features_matrix.pkl', 'wb') as f:
    pickle.dump(item_features_matrix, f)

# Optionally save interaction matrix and weights too
with open('interactions_matrix.pkl', 'wb') as f:
    pickle.dump(interactions_matrix, f)

with open('sample_weights.pkl', 'wb') as f:
    pickle.dump(weights,f)

In [ ]:
import pickle

# Load LightFM model
with open('lightfm_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load user features matrix
with open('user_features_matrix.pkl', 'rb') as f:
    user_features_matrix = pickle.load(f)

# Load item features matrix
with open('item_features_matrix.pkl', 'rb') as f:
    item_features_matrix = pickle.load(f)

# Load interaction matrix
with open('interactions_matrix.pkl', 'rb') as f:
    interactions_matrix = pickle.load(f)

# Load sample weights
with open('sample_weights.pkl', 'rb') as f:
    weights = pickle.load(f)


In [ ]:
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

# Basic evaluation (Precision@K and AUC)
precision = precision_at_k(model, interactions_matrix, k=10,
                           user_features=user_features_matrix,
                           item_features=item_features_matrix).mean()

auc = auc_score(model, interactions_matrix,
                user_features=user_features_matrix,
                item_features=item_features_matrix).mean()

# Intralist-Diversity : pairwise cosine dissimilarity of recommended items
def compute_diversity(model, item_features_matrix, top_k=10):
    item_embeddings = model.item_embeddings
    item_norm = item_embeddings / np.linalg.norm(item_embeddings, axis=1, keepdims=True) # Normalizing
    similarity = cosine_similarity(item_norm)
    diversity = []

    for user_id in range(interactions_matrix.shape[0]):
        unseen_items = get_unseen_items(user_id, interactions_matrix)
        scores = model.predict(user_id, unseen_items,
                               user_features=user_features_matrix,
                               item_features=item_features_matrix)
        top_items = unseen_items[np.argsort(-scores)[:top_k]]

        # Diversity: 1 - avg cosine similarity among top_items
        sim_matrix = similarity[np.ix_(top_items, top_items)]
        avg_sim = (np.sum(sim_matrix) - top_k) / (top_k * (top_k - 1))
        diversity.append(1 - avg_sim)

    return np.mean(diversity)

diversity_score = compute_diversity(model, item_features_matrix, top_k=10)

# Serendipity: reward unusual but relevant items
def compute_serendipity(model, interactions_matrix, top_k=10):
    serendipity = []
    item_popularity = np.array(interactions_matrix.sum(axis=0)).flatten()

    for user_id in range(interactions_matrix.shape[0]):
        unseen_items = get_unseen_items(user_id, interactions_matrix)
        scores = model.predict(user_id, unseen_items,
                               user_features=user_features_matrix,
                               item_features=item_features_matrix)
        top_items = unseen_items[np.argsort(-scores)[:top_k]]

        pop_scores = item_popularity[top_items]

        # Calculating Serendipity Score
        serendipity_score = 1 - np.mean(pop_scores / (pop_scores.max() + 1e-10))
        serendipity.append(serendipity_score)

    return np.mean(serendipity)

serendipity_score = compute_serendipity(model, interactions_matrix, top_k=10)

# Print evaluation
print(f"Precision@10: {precision:.4f}")
print(f"AUC Score: {auc:.4f}")
print(f"Diversity Score: {diversity_score:.4f}")
print(f"Serendipity Score: {serendipity_score:.4f}")

Precision@10: 0.6870
AUC Score: 0.8722
Diversity Score: 0.8058
Serendipity Score: 0.1778
